In [1]:
#from __future__ import division
import pandas as pd
import re
import numpy as np
import datetime

In [2]:
import urllib
import urllib.request

In [3]:
from bs4 import BeautifulSoup 

In [4]:
def download_csv(url, file):
    urllib.request.urlretrieve(url, file)

In [5]:
def read_stock_csv(surl):
    df = pd.read_csv(surl, parse_dates=1)
    df['Avg Price'] = ((df['High'] + df['Low']) / 2).round(4)
    return df

In [6]:
def read_div_csv(durl):
    df = pd.read_csv(durl, parse_dates=1)
    df['Avg Price'] = ((df['High'] + df['Low']) / 2).round(4)
    return df

In [7]:
# extract the dividend amount from text field
def ext_div(text):
    loc = str(text).find('HKD')
    if loc != -1:
        return float(text[loc+4:loc+4+6])
    else:
        return text

In [8]:
durl = 'http://www.aastocks.com/en/stocks/analysis/dividend.aspx?symbol=00388'
page = urllib.request.urlopen(durl)
soup = BeautifulSoup(page, 'lxml')

# web scraping
data = []
table = soup.find('table', attrs={'class':'cnhk-cf tblM s4 s5'})
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

# convert to dataframe
df = pd.DataFrame(data[1:], columns=data[0])
# sort by Ex-Date
#df.sort_values(by='Ex-Date', ascending=False, inplace=True)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [9]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#"> <head id="Head1"><meta content="IE=Edge" http-equiv="X-UA-Compatible"/> <script type="text/javascript">
var _gaq = _gaq || [];
_gaq.push(['_setAccount', 'UA-20790503-3']);
_gaq.push(['_setDomainName', 'www.aastocks.com']);
_gaq.push(['_trackPageview']);
_gaq.push(['_trackPageLoadTime']);
function OA_show(name) {
}
</script> <link href="/en/resources/style/stock_std_eng?v=RrQBXCeQ9bAAbW4UDffOrZVcVoGShYNQNCZb2fgITeo1" rel="stylesheet"/> <!--[if IE]> <link type="text/css" href='/en/resources/style/ie.css' rel="stylesheet" /> <![endif]--> <!--[if IE 6]> <link type="text/css" href='/en/resources/style/ie6.css' rel="stylesheet" /> <![endif]--> <!--[if IE 8]> <style type="text/css">
#topPanel .menu_item > a > span { letter-spacing:0px; }
</style> <![endif]-

In [ ]:
df['Particular'] = df['Particular'].apply(ext_div)
a = df[df['Type'].str.contains('Cash')]

In [ ]:
a.head()

In [ ]:
a

In [ ]:
earliest = results[0]
latest = results[len(results)-1]

In [ ]:
earliest

In [ ]:
temp = 0
accumulated = 0

for r in results:
    accumulated += r[1]
    b = a[(df['Ex-Date'] >= r[0]) & (df['Ex-Date'] >= earliest[0])]
    temp += np.sum(b['Particular']) * accumulated
    

In [ ]:
temp

In [ ]:
def months_between(start, end):
    start_year = int(start[:start.find('-')])
    start_month = int(start[start.find('-')+1:])
    end_year = int(end[:end.find('-')])
    end_month = int(end[end.find('-')+1:])
    
    year_diff = end_year - start_year
    month_diff = end_month - start_month
    
    diff = []
    
    if year_diff <= 0:
        for j in range(start_month, end_month + 1):
            diff.append(str(start_year).rjust(2, '0') + '-' + str(j).rjust(2, '0'))
    else:
        for i in range(start_year, end_year + 1):
            if (i == start_year ):
                for j in range(start_month, 12 + 1):
                    diff.append(str(i).rjust(2, '0') + '-' + str(j).rjust(2, '0'))
            elif (i == end_year):
                for j in range(1, end_month + 1):
                    diff.append(str(i).rjust(2, '0') + '-' + str(j).rjust(2, '0'))
            else:
                for j in range(1, 12 + 1):
                    diff.append(str(i).rjust(2, '0') + '-' + str(j).rjust(2, '0'))                    
      
    return diff

In [ ]:
def buy_shares(results, contrib, fee):
    shares = []
    total_shares = 0
    total_contrib = 0
    f = 0
    
    for r in results:
        s = round((contrib - fee)/r[1], 2)
        shares.append([r[0], s])
        total_shares += s
        total_contrib += contrib
        f += fee
    return shares, total_shares, total_contrib, f

In [ ]:
#def cal_dividend(shares, div):
    

In [ ]:
def cal_profit(total_shares, total_contrib, current_price, dividend=0):
    profit = (current_price * total_shares + dividend - total_contrib) / total_contrib
    
    return profit    

In [ ]:
def eval_stock(surl, durl, contrib, fee, start, end, day=10):
    # contrib: monthly contribution amount
    # fee
    # current_price
    # start: a string indicate the start year and month
    # end: a string indicate the end year and month
    # day: day to execution    
    
    results = []
    
    months = months_between(start, end)
    
    df = read_stock_csv(surl)
    
    # current price is the latest average price
    current_price = df[df['Date'] ==np.max(df['Date'])].iloc[0]['Avg Price']
    
    for m in months:
        for d in range(day, day + 5):
            temp = df[df['Date'].str.contains(m + '-' + str(d).rjust(2, '0'))]
            if len(temp) > 0:
                results.append([m + '-' + str(d).rjust(2, '0'), temp.iloc[0]['Avg Price']])
                break
                
    shares, total_shares, total_contrib, total_fee = buy_shares(results, contrib, fee)
    
    profit = cal_profit(total_shares, total_contrib, current_price)
                    
    return shares

In [ ]:
#stocks = ['0002', '0175', '0700', '0388', '0066']
stocks = ['0001']

In [ ]:
today = datetime.datetime.now().timetuple()

# download csv files
for s in stocks:
    surl = 'http://real-chart.finance.yahoo.com/table.csv?s='+s+'.HK'+'&a=05&b=16&c=2000&d=03&e='+str(today[2])+'&f=2016&g=d&ignore=.csv'
    download_csv(surl, s+'.csv')
#http://real-chart.finance.yahoo.com/table.csv?s=2800.HK&a=11&b=31&c=2000&d=03&e=26&f=2016&g=v&ignore=.csv    

In [ ]:
#results = []
for s in stocks:
    surl = s+'.csv'
    durl = ''
    #results.append([s, eval_stock(surl, durl, 2000, 50, '2015-01', '2016-04')])
    results = eval_stock(surl, durl, 2000, 50, '2015-01', '2016-04')
results